<a href="https://colab.research.google.com/github/DayoungKwon/pko-t5/blob/main/chat-alpaca-ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install transformers
!pip install wandb
!pip install datasets
!pip install accelerate

In [ ]:
!mkdir -p data_in/KOR/naver_movie
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt \
              -O data_in/KOR/naver_movie/ratings_train.txt
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt \
              -O data_in/KOR/naver_movie/ratings_test.txt

In [6]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import pandas as pd
import numpy as np
import re

import random
from random import sample

from tqdm import tqdm

In [7]:
SEED_NUM = 1234
np.random.seed(SEED_NUM)
random.seed(SEED_NUM)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")
cls_model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot",
                                                 torch_dtype=torch.float16,
                                                 low_cpu_mem_usage=True).cuda()

In [ ]:
cls_model.config.max_length = 2048
cls_model.config.pad_token_id = 0

In [ ]:
DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = './data_out/KOR'

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_train.txt')
DATA_TEST_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_test.txt')

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0, delimiter = '\t', quoting = 3)
train_data = train_data.dropna()

In [ ]:
print('데이터 positive 라벨: ', '긍정')
print('데이터 negative 라벨: ', '부정')

In [ ]:
print('학습 예시 케이스 구조: ', '문장: 오늘 기분이 좋아\n감정: 긍정\n')
print('gpt 최대 토큰 길이: ', cls_model.config.max_position_embeddings)


In [ ]:
sent_lens = [len(tokenizer(s).input_ids) for s in tqdm(train_data['document'])]

print('Few shot 케이스 토큰 평균 길이: ', np.mean(sent_lens))
print('Few shot 케이스 토큰 최대 길이: ', np.max(sent_lens))
print('Few shot 케이스 토큰 길이 표준편차: ',np.std(sent_lens))
print('Few shot 케이스 토큰 길이 80 퍼센타일: ',np.percentile(sent_lens, 80))

In [ ]:
train_fewshot_data = []

for train_sent, train_label in tqdm(train_data[['document', 'label']].values):
    tokens = tokenizer(train_sent).input_ids

    if len(tokens) <= 25:
        train_fewshot_data.append((train_sent, train_label))

In [ ]:
test_data = pd.read_csv(DATA_TEST_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

In [ ]:
# Full Dataset
# sample_size = len(test_data)

# Sampled Dataset
sample_size = 500

train_fewshot_samples = []

for _ in range(sample_size):
    fewshot_examples = sample(train_fewshot_data, 10)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=SEED_NUM)